In [ ]:
import numpy as np
import pandas
from minigo import dao
from glob import glob
#[sorted(choice), 1 - drop, drop, meandrop, a, fitloss]
ret = {}
for npy in glob('*_n.npy'):
    x = pandas.DataFrame(np.load(npy, allow_pickle=True))
    df = pandas.read_csv(npy.replace('_n.npy', '_d.csv'), index_col=0)
    N = df.shape[1]
    print(x[x[4]>0.7/N].shape)
    x = x[x[4]>0.7/N].sort_values(5);
    rs = []
    #rs2= []
    for a in x.values:
        dff = df.iloc[a[0]]
        d, p, _, _ = dao.get_dpr(dff)
        s = d.sum(axis=0) / d.shape[0]
        slope, b = np.polyfit(np.arange(s.shape[0]), s, 1)
        #slope2, b2 = np.polyfit(np.arange(12), s[-12:], 1)
        r = s[-1] / ((len(s) - 1) * a[4] + b) - 1
        rs.append(r)
        #rs2.append(slope2)
    rs = pandas.DataFrame(rs)
    rs.index = x.index
    rs.columns = ['now']
    #rs2 = pandas.DataFrame(rs2)
    #rs2.index = x.index
    #rs2.columns = ['recent_slope']
    x = pandas.concat([x, rs, rs2], axis=1)
    x = x[x['now'] < -0.01].iloc[0]
    #x = x.iloc[0]
    #x = x[x['recent_slope'] > 0.6/N].iloc[0]
    
    #.iloc[0]
    
    stocks = df.index[x[0]].tolist()
    dff = df.iloc[x[0]]
    day = npy.replace('_n.npy', '')
    d, p, _, _ = dao.get_dpr(dff)
    ret[day] = [list(map(lambda a: '@"' + a.split(' ')[1][1:-2] + '"@', stocks)), (p / p.sum()).tolist(), x[0], x[2], x[4]]

In [ ]:
str(ret).replace('\'@', '').replace('@\'', '').replace('\'', '"')

In [ ]:
def demo(day):
    df = pandas.read_csv(day + '_d.csv', index_col=0)
    d,p,_,_=dao.get_dpr(df.iloc[ret[day][2]])
    s = d.sum(axis=0) / 10
    stat = dao.stat(s)
    print(day, stat[0], stat[2] * len(s) / s[0] * 100, stat[3])
    _, b = np.polyfit(np.arange(s.shape[0]), s, 1)
    s2 = np.arange(len(s)) * stat[2] + b
    pandas.DataFrame([s,s2]).T.plot(title=day)

for x in ret:
    demo(x)

In [ ]:
df = pandas.read_csv('2008-07-07_d.csv', index_col=0)
x = pandas.DataFrame(np.load('lasttime.npy', allow_pickle=True)).sort_values(5).iloc[:100,:].sort_values(4, ascending=False).iloc[0]
d,p,_,_=dao.get_dpr(df.iloc[x[0]])

In [ ]:
s = d.sum(axis=0)
pandas.DataFrame(s).plot()
dao.stat(d.sum(axis=0))

In [ ]:
x = pandas.DataFrame(np.load('2008-07-07_n.npy', allow_pickle=True))
x[x[2]<1].shape[0]


    

In [ ]:
d,eligible = dao.get_d('2008-07-07_d.csv')
BLACK_LIST = []
MAX=10

def find_eligible_no_network(got):
    if len(got) == 0: return list(range(d.shape[0]))
    origin = eligible[got[0]]
    for i in range(1, len(got)):
        origin = np.logical_and(eligible[got[i]], origin)
    ret = np.where(origin)[0].tolist()
    return [i for i in ret if i not in BLACK_LIST]

def factorial_random_no_network(gots, repeat):
    if gots is None or len(gots) == 0:
        gots = []
        f = find_eligible_no_network([])
        for j in range(repeat):
            c = np.random.choice(f, 1)[0]
            gots.append([c])

    toContinue = []
    toKeep = []
    for got in gots:
        if len(got) == MAX:
            toKeep.append(got)
        else:
            toContinue.append(got)

    if len(toContinue) == 0:
        return toKeep

    fines = []
    for got in toContinue:
        fines.append(find_eligible_no_network(got))
        
    nextGots = []
    for i in range(len(toContinue)):
        for j in range(repeat):
            if len(fines[i]) > 0:
                c = np.random.choice(fines[i], 1)[0]
                nextGots.append(toContinue[i] + [c])

    return factorial_random_no_network(toKeep + nextGots, repeat)

result = []

for i in range(100):
    xs = factorial_random_no_network([], 2)
    for x in xs:
        x.sort()
        s = d[x].sum(axis=0) / MAX
        result.append(list(dao.stat(s)) + [x])

In [ ]:
result = pandas.DataFrame(result)
result[result[0] < 2].sort_values(0)

In [ ]:
dao.playByRandom('abc')